# Spatial TX registration to slab

1. Run st2block to generate affine transforms between st and blockface images. The outputs of these affines are specified in `transforms_path`
2. Obtain block2slab transforms  
    a. In this case, it was performed with SVG
3. Create dict of slab_id to slab_image



In [1]:
from hmba_stx_registration import get_metadata_df, Specimen, process_barcode
from hmba_stx_registration.st_to_slab import get_bf_affines_from_svg
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime

date=datetime.now().strftime("%m%d%Y")
base_path = Path("/home/mike/workspace/data/macaque_brain/QM24.50.002")
sptx_path = base_path / "sptx"
barcodes_path = sptx_path / "xenium"
transforms_path = sptx_path / "st2block_output_02162026"
um_per_px = 20
mm_per_px = um_per_px / 1000
metadata_df = get_metadata_df(barcodes_path)
slab_ids = range(45, 54)

/home/mike/workspace/data/macaque_brain/QM24.50.002/sptx/xenium/QM24.50.002_section_metadata.csv is not a directory, skipping


In [3]:
bf_affines = {}
slab_imgs = {}
for slab_id in slab_ids:
    slab_imgs[slab_id] = plt.imread(base_path / "slabs" / "combined_stack" / f"CX{slab_id}_a_frozen.png")
    svg_path = base_path / f"QM24.50.002.CX.{slab_id}.svg"
    if not svg_path.exists():
        print(f"Missing SVG: {svg_path}")
        continue
    bf_affines_slab = get_bf_affines_from_svg(svg_path, base_path)
    bf_affines.update(bf_affines_slab)

In [ ]:
barcodes = metadata_df['barcode'].tolist()
for barcode in barcodes:
    specimen = Specimen(barcode, barcodes_path)
    process_barcode(
        specimen,
        bf_affines,
        slab_imgs,
        transforms_path,
        um_per_px,
        date,
        version="0.1",
        sync_to_s3=False,
        sync_dryrun=True,
    )